In [6]:
# Cell 1: Import and setup
import re
from typing import Any

# Cell 2: Parse expression function
def parse_expression(expr: str):
    """Parse expressions like $.foo == "some value" into field name and comparison value."""
    # Match pattern like $.fieldname == "value" or $.fieldname == 123
    pattern = r'\$\.([\w\.]+)\s*(==|!=|>|<|>=|<=)\s*(.+)'
    match = re.match(pattern, expr.strip())

    if not match:
        raise ValueError(f"Invalid expression format: {expr}")

    field, operator, value = match.groups()

    # Try to evaluate the value part (handles quoted strings, numbers, etc.)
    try:
        evaluated_value = eval(value)
    except:
        evaluated_value = value  # Keep as string if eval fails

    return field, operator, evaluated_value

# Cell 3: Test the parser
test_expressions = [
    '$.foo == "some value"',
    '$.nested.field == 123',
    '$.active == True',
    '$.count > 5'
]

for expr in test_expressions:
    field, op, value = parse_expression(expr)
    print(f"Expression: {expr}")
    print(f"  Field: {field}")
    print(f"  Operator: {op}")
    print(f"  Value: {value} (type: {type(value).__name__})")
    print()

# Cell 4: Implement expression evaluation
def evaluate_expression(expr: str, obj: dict[str, Any]) -> bool:
    """Evaluate expressions like $.foo == "some value" against an object."""
    import pydash

    field, operator, value = parse_expression(expr)
    field_value = pydash.get(obj, field)

    if operator == "==":
        return field_value == value
    elif operator == "!=":
        return field_value != value
    elif operator == ">":
        return field_value > value
    elif operator == "<":
        return field_value < value
    elif operator == ">=":
        return field_value >= value
    elif operator == "<=":
        return field_value <= value
    else:
        raise ValueError(f"Unsupported operator: {operator}")

# Cell 5: Test the evaluator
test_object = {
    "foo": "some value",
    "nested": {"field": 123},
    "active": True,
    "count": 10
}

for expr in test_expressions:
    result = evaluate_expression(expr, test_object)
    print(f"Expression: {expr}")
    print(f"  Result: {result}")
    print()

# Cell 6: Modified Expr.test implementation
class Expr:
    def __init__(self, args: str):
        self.args = args

    def test(self, o: dict[str, Any]) -> bool:
        return evaluate_expression(self.args, o)

# Cell 7: Test the modified Expr class
expr_instance = Expr('$.foo == "some value"')
result = expr_instance.test(test_object)
print(f"Expression: {expr_instance.args}")
print(f"Result: {result}")

expr_instance2 = Expr('$.count > 5')
result2 = expr_instance2.test(test_object)
print(f"Expression: {expr_instance2.args}")
print(f"Result: {result2}")

Expression: $.foo == "some value"
  Field: foo
  Operator: ==
  Value: some value (type: str)

Expression: $.nested.field == 123
  Field: nested.field
  Operator: ==
  Value: 123 (type: int)

Expression: $.active == True
  Field: active
  Operator: ==
  Value: True (type: bool)

Expression: $.count > 5
  Field: count
  Operator: >
  Value: 5 (type: int)

Expression: $.foo == "some value"
  Result: True

Expression: $.nested.field == 123
  Result: True

Expression: $.active == True
  Result: True

Expression: $.count > 5
  Result: True

Expression: $.foo == "some value"
Result: True
Expression: $.count > 5
Result: True
